In [4]:
!ls {spark_version}-bin-hadoop3.tgz

ls: cannot access 'spark-3.4.0-bin-hadoop3.tgz': No such file or directory


In [5]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (172 kB/s)
Reading package lists... Done


In [25]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
file_path = "file://" + SparkFiles.get("home_sales_revised.csv")

# Read data into a DataFrame
home_sales_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [39]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

spark.sql("SELECT * FROM home_sales_view").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [40]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

average_price_query = """
SELECT
    DISTINCT year(date) as sale_year,
    ROUND(AVG(CASE WHEN bedrooms = 4 THEN price ELSE null END), 2) as avg_price
FROM
    home_sales
GROUP BY
    sale_year
ORDER BY
    sale_year
"""

# Execute the query
average_price_result = spark.sql(average_price_query)

# Show the result
average_price_result.show()

+---------+---------+
|sale_year|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [41]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_price_query = """
SELECT
    date_built,
    ROUND(AVG(CASE WHEN bedrooms = 3 AND bathrooms = 3 THEN price ELSE null END), 2) as avg_price
FROM
    home_sales
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
average_price_result = spark.sql(average_price_query)

# Show the result
average_price_result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [42]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

average_price_query = """
SELECT
    date_built,
    ROUND(AVG(CASE WHEN bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 THEN price ELSE null END), 2) as avg_price
FROM
    home_sales
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
average_price_result = spark.sql(average_price_query)

# Show the result
average_price_result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [43]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SparkSQL query to calculate the average "view" rating for homes with a price greater than or equal to $350,000
average_view_query = """
SELECT
    ROUND(AVG(view), 2) as avg_view_rating
FROM
    home_sales
WHERE
    price >= 350000
"""

# Execute the query
average_view_result = spark.sql(average_view_query)

# Show the result
average_view_result.show()



print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.353853702545166 seconds ---


In [44]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("home_sales")

# Check if the table is cached
print("Is home_sales_view cached:", spark.catalog.isCached("home_sales"))


Is home_sales_view cached: True


In [45]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [48]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time_cached = time.time()

# SparkSQL query with cached data to filter out view ratings with average price >= $350,000
filtered_view_query_cached = """
SELECT
    ROUND(AVG(view), 2) as avg_view_rating
FROM
    home_sales
WHERE
    price >= 350000
"""

# Execute the query with cached data
filtered_view_result_cached = spark.sql(filtered_view_query_cached)

# Show the result
filtered_view_result_cached.show()

# Print the runtime with cached data
print("--- Cached Runtime: %s seconds ---" % (time.time() - start_time_cached))


# Uncache the table
spark.catalog.uncacheTable("home_sales_view")

# Start the timer for uncached data
start_time_uncached = time.time()

# SparkSQL query without cached data to filter out view ratings with average price >= $350,000
filtered_view_query_uncached = """
SELECT
    ROUND(AVG(view), 2) as avg_view_rating
FROM
    home_sales
WHERE
    price >= 350000
"""

# Execute the query without cached data
filtered_view_result_uncached = spark.sql(filtered_view_query_uncached)

# Show the result
filtered_view_result_uncached.show()

# Print the runtime without cached data
print("--- Uncached Runtime: %s seconds ---" % (time.time() - start_time_uncached))


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Cached Runtime: 0.3128805160522461 seconds ---
+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Uncached Runtime: 0.40084028244018555 seconds ---


In [49]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
partitioned_parquet_path = "path/to/partitioned_home_sales"

# Write the DataFrame to partitioned Parquet format
home_sales_df.write.partitionBy("date_built").parquet(partitioned_parquet_path)

In [50]:
# 11. Read the parquet formatted data.

partitioned_home_sales_df = spark.read.parquet(partitioned_parquet_path)

# Show the first few rows of the DataFrame
partitioned_home_sales_df.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [51]:
# 12. Create a temporary table for the parquet data.
partitioned_home_sales_df.createOrReplaceTempView("partitioned_home_sales")

In [55]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time_parquet = time.time()

# SparkSQL query with Parquet DataFrame to filter out view ratings with average price >= $350,000
filtered_view_query_parquet = """
SELECT
    ROUND(AVG(view), 2) as avg_view_rating
FROM
    partitioned_home_sales
WHERE
    price >= 350000
"""

# Execute the query with Parquet DataFrame
filtered_view_result_parquet = spark.sql(filtered_view_query_parquet)

# Show the result
filtered_view_result_parquet.show()

# Print the runtime with Parquet DataFrame
print("--- Parquet Runtime: %s seconds ---" % (time.time() - start_time_parquet))


# Start the timer for cached original DataFrame
start_time_cached_original = time.time()

# SparkSQL query with cached original DataFrame to filter out view ratings with average price >= $350,000
filtered_view_query_cached_original = """
SELECT
    ROUND(AVG(view), 2) as avg_view_rating
FROM
    home_sales_view
WHERE
    price >= 350000
"""

# Execute the query with cached original DataFrame
filtered_view_result_cached_original = spark.sql(filtered_view_query_cached_original)

# Show the result
filtered_view_result_cached_original.show()

# Print the runtime with cached original DataFrame
print("--- Cached Original Runtime: %s seconds ---" % (time.time() - start_time_cached_original))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Parquet Runtime: 0.26195240020751953 seconds ---
+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Cached Original Runtime: 0.5316915512084961 seconds ---


In [56]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [57]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False